Need to build the Transversional CNOT and CNOT gate

This let's use create logical CNOT gate 

Implement a logical CNOT as applying a CNOT on each corresponding physical qubit pair after encoding. 

Encode block A (control) and Encode block B (target)

CNOTc→t​=(I⊗H)CZ(I⊗H). H gate on target before, CZ control to target, H gate after on target



Process flow : 

- Encoding circuit (logical qubit) 
    - 7 qubit color/Steane state injection [[7,1,3]] 

- QEC (Steane) to create memory 

Encode Circuit (logical qubit)

![Encoding Circuit](Encode_circ_(logical_qubit).png)


Distillation circuit 

![Distillation Circuit](distillation_d3.png)

Used later (if at all)

In [ ]:
# CNOT Gate between two logical qubits encoded with the Steane code
# 
# 
# import math

# def H(q): # H gate up to global phase
#     squin.u3(math.pi/2, 0.0, math.pi, q)

# def logical_cnot_blockwise(qc, qt):   # qc, qt are length-7 lists of qubits (encoded blocks)
#     for i in range(7):
#         H(qt[i])
#         squin.cz(qc[i], qt[i])
#         H(qt[i])

In [2]:
### Distillation Code ###

#call Logical qubit (encoded / error corrected) 

#squin.cz(q[1], q[2])
        # squin.sqrt_x(q[0])
        # squin.sqrt_x(q[1])
        # squin.sqrt_x(q[4])
        # squin.cz(q[0],q[1])
        # squin.cz(q[2],q[3])
        # squin.sqrt_y(q[0])
        # squin.sqrt_y(q[3])
        # squin.cz(q[0],q[2])
        # squin.cz(q[3],q[4])
        # squin.sqrt_x_adj(q[0]) 
        # squin.cz(q[1],q[3])
        # squin.cz(q[0],q[4])
        # for i in range(5):
        #    squin.sqrt_x_adj(q[i])

In [3]:
from bloqade import squin
from kirin.dialects.ilist import IList
import numpy as np
import bloqade.stim
import bloqade.tsim

In [7]:
def MSD_encoding(theta, phi,basis="z"):

    @squin.kernel
    def parameterized_MSD_encoding():
        q = squin.qalloc(7)  # allocate qubits
        squin.u3(theta,phi,0,q[6])
        for i in range(6):
            squin.sqrt_y_adj(q[i])
        # [squin.broadcast.sqrt_y_adj(q[i]) for i in range(5)]
        squin.cz(q[1], q[2])
        squin.cz(q[3], q[4])
        squin.cz(q[5], q[6])
        squin.sqrt_y(q[6])
        squin.cz(q[0],q[3])
        squin.cz(q[2],q[5])
        squin.cz(q[4],q[6])
        for i in range(5):
            squin.sqrt_y(q[i+2])
        # [squin.broadcast.sqrt_y(q[i+2]) for i in range(5)]
        squin.cz(q[0],q[1])
        squin.cz(q[2],q[3])
        squin.cz(q[4],q[5])
        squin.sqrt_y(q[1])
        squin.sqrt_y(q[2])
        squin.sqrt_y(q[4])
    
    return parameterized_MSD_encoding

In [8]:
def error_detection(theta, phi):
    @squin.kernel 
    def parameterized_error_detection():
        MSD_enc = MSD_encoding(theta, phi)
        z_ancilla = MSD_encoding((np.pi)/2, 0) # Injected state is |+>
        z_measurement = []

        for i in range(7):
            squin.cx(MSD_enc.q[i], z_ancilla.q[i])
            z_measurement.append(squin.measure(z_ancilla.q[i]))
            # print(z_measurement[i])
        
        x_ancilla = MSD_encoding(0, 0) # Injected state is |0>
        x_measurement = []
        for i in range(7):
            squin.cx(x_ancilla.q[i], MSD_enc.q[i])
            squin.h(x_ancilla.q[i])
            squin.measure(x_ancilla.q[i])
            x_measurement.append(squin.measure(x_ancilla.q[i]))
            # print(x_measurement[i])
    return parameterized_error_detection

In [9]:
def MSD_encoding(theta, phi,basis="z"):
    @squin.kernel
    def z_MSD_encoding(q_subset): # MSD encoding of Z-basis ancilla (|+>); equivalent to the line z_ancilla = MSD_encoding((np.pi)/2, 0) before
        squin.u3((np.pi)/2,0,0,q_subset[6])
        for i in range(6):
            squin.sqrt_y_adj(q_subset[i])
        # [squin.broadcast.sqrt_y_adj(q_subset[i]) for i in range(5)]
        squin.cz(q_subset[1], q_subset[2])
        squin.cz(q_subset[3], q_subset[4])
        squin.cz(q_subset[5], q_subset[6])
        squin.sqrt_y(q_subset[6])
        squin.cz(q_subset[0],q_subset[3])
        squin.cz(q_subset[2],q_subset[5])
        squin.cz(q_subset[4],q_subset[6])
        for i in range(5):
            squin.sqrt_y(q_subset[i+2])
        # [squin.broadcast.sqrt_y(q_subset[i+2]) for i in range(5)]
        squin.cz(q_subset[0],q_subset[1])
        squin.cz(q_subset[2],q_subset[3])
        squin.cz(q_subset[4],q_subset[5])
        squin.sqrt_y(q_subset[1])
        squin.sqrt_y(q_subset[2])
        squin.sqrt_y(q_subset[4])
    @squin.kernel
    def parameterized_MSD_encoding(): # MSD encoding of both injected state and Z-basis ancilla
        q = squin.qalloc(14)  # allocate qubits
        squin.u3(theta,phi,0,q[6])
        for i in range(6):
            squin.sqrt_y_adj(q[i])
        # [squin.broadcast.sqrt_y_adj(q[i]) for i in range(5)]
        squin.cz(q[1], q[2])
        squin.cz(q[3], q[4])
        squin.cz(q[5], q[6])
        squin.sqrt_y(q[6])
        squin.cz(q[0],q[3])
        squin.cz(q[2],q[5])
        squin.cz(q[4],q[6])
        for i in range(5):
            squin.sqrt_y(q[i+2])
        # [squin.broadcast.sqrt_y(q[i+2]) for i in range(5)]
        squin.cz(q[0],q[1])
        squin.cz(q[2],q[3])
        squin.cz(q[4],q[5])
        squin.sqrt_y(q[1])
        squin.sqrt_y(q[2])
        squin.sqrt_y(q[4])
        z_MSD_encoding([q[7], q[8], q[9], q[10], q[11], q[12], q[13]]) # Adding the Z-basis ancilla encoding
    
    return parameterized_MSD_encoding

In [10]:
def MSD_encoding_plus_ancilla(theta, phi,lam,basis="z"):
    @squin.kernel
    def parameterized_MSD_encoding(q_subset,theta,phi,lam):
        squin.u3(theta,phi,lam,q_subset[6])
        for i in range(6):
            squin.sqrt_y_adj(q_subset[i])
        # [squin.broadcast.sqrt_y_adj(q[i]) for i in range(5)]
        squin.cz(q_subset[1], q_subset[2])
        squin.cz(q_subset[3], q_subset[4])
        squin.cz(q_subset[5], q_subset[6])
        squin.sqrt_y(q_subset[6])
        squin.cz(q_subset[0],q_subset[3])
        squin.cz(q_subset[2],q_subset[5])
        squin.cz(q_subset[4],q_subset[6])
        for i in range(5):
            squin.sqrt_y(q_subset[i+2])
        # [squin.broadcast.sqrt_y(q[i+2]) for i in range(5)]
        squin.cz(q_subset[0],q_subset[1])
        squin.cz(q_subset[2],q_subset[3])
        squin.cz(q_subset[4],q_subset[5])
        squin.sqrt_y(q_subset[1])
        squin.sqrt_y(q_subset[2])
        squin.sqrt_y(q_subset[4])
        
    @squin.kernel
    def circ():
        q=squin.qalloc(14)
        #blah([q[0],q[1]])
        parameterized_MSD_encoding([q[0],q[1],q[2],q[3],q[4],q[5],q[6]],theta=theta,phi=phi,lam=lam)
        parameterized_MSD_encoding([q[7],q[8],q[9],q[10],q[11],q[12],q[13]],theta=np.pi/2,phi=0,lam=np.pi)
        
    
    return circ

In [ ]:

# --- 1) Encode helper ---

def encode_into(theta, phi,lam,basis="z"):
    @squin.kernel 
    squin.u3(theta, phi, 0, qb[6]) # qb is length-7 list/slice: qb[0]..qb[6]
    for i in range(6):
        squin.sqrt_y_adj(qb[i])

    squin.cz(qb[1], qb[2])
    squin.cz(qb[3], qb[4])
    squin.cz(qb[5], qb[6])
    squin.sqrt_y(qb[6])

    squin.cz(qb[0], qb[3])
    squin.cz(qb[2], qb[5])
    squin.cz(qb[4], qb[6])

    for i in range(5):
        squin.sqrt_y(qb[i+2])

    squin.cz(qb[0], qb[1])
    squin.cz(qb[2], qb[3])
    squin.cz(qb[4], qb[5])

    squin.sqrt_y(qb[1])
    squin.sqrt_y(qb[2])
    squin.sqrt_y(qb[4])


def MSD_encoding(theta, phi):
    @squin.kernel
    def k():
        q = squin.qalloc(7)
        encode_into(q, theta, phi)
    return k

# --- 2) Transversal logical ops on 7q blocks ---
def L_sqrt_x(block):
    for k in range(7):
        squin.sqrt_x(block[k])

def L_sqrt_x_adj(block):
    for k in range(7):
        squin.sqrt_x_adj(block[k])

def L_sqrt_y(block):
    for k in range(7):
        squin.sqrt_y(block[k])

def L_cz(blockA, blockB):
    # transversal CZ (pair physical qubits by index)
    for k in range(7):
        squin.cz(blockA[k], blockB[k])

# --- 3) Distillation kernel: 5 logical qubits => 5 blocks of 7 physical qubits ---
def distillation_5to1(theta, phi):
    @squin.kernel
    def circuit():
        q = squin.qalloc(35)
        blocks = [q[7*i:7*(i+1)] for i in range(5)]  # blocks[0]..blocks[4], each length 7

        # Encode all 5 logical inputs (same |psi(theta,phi)> for now)
        for b in range(5):
            encode_into(blocks[b], theta, phi)

        # --- Your logical-level distillation circuit, mapped transversally ---
        # (Mapping your commented snippet where q[0]..q[4] were logical qubits)
        L_cz(blocks[1], blocks[2])

        L_sqrt_x(blocks[0])
        L_sqrt_x(blocks[1])
        L_sqrt_x(blocks[4])

        L_cz(blocks[0], blocks[1])
        L_cz(blocks[2], blocks[3])

        L_sqrt_y(blocks[0])
        L_sqrt_y(blocks[3])

        L_cz(blocks[0], blocks[2])
        L_cz(blocks[3], blocks[4])

        L_sqrt_x_adj(blocks[0])

        L_cz(blocks[1], blocks[3])
        L_cz(blocks[0], blocks[4])

        for i in range(5):
            L_sqrt_x_adj(blocks[i])

    return circuit


Cannot lower ListComp node: <ast.ListComp object at 0x123165310>


In [ ]:
distill_kernel = distillation_5to1(theta=0.0, phi=0.0)
tsim_circ = bloqade.tsim.Circuit(distill_kernel)
tsim_circ.diagram(height=500)
